# Functions

#### Function library for financial calculation:
- Present Value (PV)
- Future Value (FV)
- Net Present Value (NPV)
- Internal Rate of Return (IRR)

In [25]:
# Importing numpy_financial for the necessary financial functions
import numpy_financial as npf

def PV(rate, nper, pmt, fv, when):
    """
    Function to calculate Present Value (PV).

    The Present Value (PV) is the current value of a series of future cash flows,
    given a specific interest rate.

    Parameters:
    rate (float): The interest rate per period.
    nper (int): The number of periods.
    pmt (float): The payment made each period (should be negative if it’s an outgoing payment).
    fv (float): The future value, or a cash balance you want to attain after the last payment is made.
    when (str): The timing of the payment ('begin' for beginning of the period, 'end' for end of the period).

    Returns:
    float: The present value of the series of payments.
    """
    # Calculate Present Value
    return npf.pv(rate, nper, pmt, fv=fv, when=when)


def FV(rate, nper, pmt, pv, when):
    """
    Function to calculate Future Value (FV).

    The Future Value (FV) is the amount of money that you will have at the end of a certain
    number of periods, considering the present value, the rate of return, and periodic payments.

    Parameters:
    rate (float): The interest rate per period.
    nper (int): The number of periods.
    pmt (float): The payment made each period (should be negative if it’s an outgoing payment).
    pv (float): The present value, or the initial investment.
    when (str): The timing of the payment ('begin' for beginning of the period, 'end' for end of the period).

    Returns:
    float: The future value of the investment after the specified number of periods.
    """
    # Calculate Future Value
    return npf.fv(rate, nper, pmt, pv, when=when)


def NPV(rate, values):
    """
    Function to calculate Net Present Value (NPV).

    Net Present Value (NPV) is a method of calculating the present value of all future
    cash flows (positive and negative) of an investment, discounted at a specified rate.

    Parameters:
    rate (float): The discount rate (rate of return).
    values (list of float): A list of cash flows for each period, where the first value is the initial investment.
    
    Returns:
    float: The net present value of the series of cash flows.
    """
    # Calculate Net Present Value
    return npf.npv(rate, values)


def IRR(values):
    """
    Function to calculate Internal Rate of Return (IRR).

    The Internal Rate of Return (IRR) is the discount rate that makes the net present value of
    all future cash flows from a particular project or investment equal to zero.

    Parameters:
    values (list of float): A list of cash flows, where the first value is the initial investment.

    Returns:
    float: The internal rate of return (IRR) for the series of cash flows.
    """
    # Calculate Internal Rate of Return
    return npf.irr(values)



In [26]:
import financial_calculations as fc

# Example usage
present_value = fc.PV(0.05, 10, -1000, 10000, 'end')
print(f"Present Value: {present_value}")


Present Value: 1582.602393777225


#### Function that calculates stock metric
- Daily returns
- Volatility
- Sharpe ratio

In [27]:
import yfinance as yf
import numpy as np
import pandas as pd
from fredapi import Fred

# Function for pulling risk-free rate of return
def get_risk_free_rate():
    """
    Fetches the most recent 10-year Treasury Yield (Risk-Free Rate) from the FRED database.
    Returns:
        float: The current risk-free rate (10-year Treasury yield) in decimal form.
    """
    # API key
    api_key = '732e3423380ec3230c311938303e797d'
    fred = Fred(api_key=api_key)
    # 'GS10' is the FRED code for the 10-year Treasury yield; convert percentage to decimal
    risk_free_rate = fred.get_series('GS10').iloc[-1] / 100.0
    return risk_free_rate

# Function for pulling stock data and calculating metrics
def analyze_stock_data(ticker_symbol):
    """
    Calculates daily logarithmic returns, 30-day rolling volatility, and Sharpe Ratio.
    
    Parameters:
        ticker_symbol (str): The stock ticker symbol (e.g., "MSFT", "AAPL")
    
    Returns:
        tuple: (log_returns, volatility, sharpe_ratio)
            - log_returns: Daily logarithmic returns.
            - volatility: 30-day rolling volatility (annualized).
            - sharpe_ratio: The Sharpe ratio of the stock.
    """
    # Increase period to "3mo" so there are enough data points for a 30-day rolling window
    stock_data = yf.download(ticker_symbol, period='3mo')
    if stock_data.empty:
        raise ValueError(f"No data fetched for ticker symbol: {ticker_symbol}")

    # Ensure that 'Close' is a Series
    close = stock_data['Close']
    if isinstance(close, pd.DataFrame):
        close = close.squeeze()

    # Calculate daily logarithmic returns
    log_returns = np.log(close / close.shift(1))
    
    # Calculate 30-day rolling volatility (annualized)
    volatility = log_returns.rolling(window=30).std() * np.sqrt(252)
    
    # Calculate annualized return and overall volatility
    annualized_return = log_returns.mean() * 252
    annualized_volatility = log_returns.std() * np.sqrt(252)

    # Fetch risk-free rate
    risk_free_rate = get_risk_free_rate()

    # Calculate Sharpe Ratio: (Return - Risk-Free Rate) / Volatility
    sharpe_ratio = (annualized_return - risk_free_rate) / annualized_volatility

    return log_returns, volatility, sharpe_ratio

# Main Program 
def main():
    """
    Main program to interact with the user, retrieve stock data, and calculate stock metrics.
    """
    ticker_symbol = input("Enter the ticker symbol of the stock (e.g., MSFT, AAPL): ").upper()
    
    try:
        log_returns, volatility, sharpe_ratio = analyze_stock_data(ticker_symbol)
        
        # Extract scalar values for printing
        avg_log_return = log_returns.mean()
        if isinstance(avg_log_return, pd.Series):
            avg_log_return = avg_log_return.iloc[0]
        
        vol_value = volatility.iloc[-1]
        if isinstance(vol_value, pd.Series):
            vol_value = vol_value.iloc[0]
        
        if isinstance(sharpe_ratio, pd.Series):
            sharpe_ratio = sharpe_ratio.iloc[0]
        
        # Print the results
        print(f"\nResults for {ticker_symbol}:")
        print(f"Average Daily Log Returns: {avg_log_return:.4f}")
        
        # Check if volatility is a valid number
        if not np.isnan(vol_value):
            print(f"30-Day Rolling Volatility (Annualized): {vol_value:.4f}")
        else:
            print("30-Day Rolling Volatility (Annualized): Data not available")
        
        print(f"Sharpe Ratio (Annualized): {sharpe_ratio:.4f}")
    
    except Exception as e:
        print(f"Error fetching stock data for {ticker_symbol}: {e}")

# Run the main program
if __name__ == "__main__":
    main()




[*********************100%***********************]  1 of 1 completed



Results for MSFT:
Average Daily Log Returns: -0.0006
30-Day Rolling Volatility (Annualized): 0.2829
Sharpe Ratio (Annualized): -0.8259


#### Bond pricing function that:
- Takes face value, coupon rate, years to maturity, and market rate
- Returns the current bond price

In [28]:
def bond_price(face_value, coupon_rate, years_to_maturity, market_rate):
    """
    Function to calculate the current price of a bond based on given parameters.

    Parameters:
    face_value (float): The face value (or par value) of the bond.
    coupon_rate (float): The annual coupon rate as a decimal (e.g., 5% as 0.05).
    years_to_maturity (int): The number of years until the bond matures.
    market_rate (float): The market interest rate (annual yield to maturity) as a decimal.

    Returns:
    float: The current price of the bond.
    """
    # Annual coupon payment (C)
    coupon_payment = coupon_rate * face_value
    
    # Price calculation using the formula
    price = (coupon_payment * (1 - (1 + market_rate)**(-years_to_maturity)) / market_rate) + (face_value / (1 + market_rate)**years_to_maturity)

    return price

# Main Program
def main():
    """
    Main program to take user input for bond parameters and calculate the bond price.
    """
    try:
        # Get user input for bond parameters
        face_value = float(input("Enter the face value of the bond (e.g., 1000): "))
        coupon_rate = float(input("Enter the coupon rate of the bond (as a decimal, e.g., 0.05 for 5%): "))
        years_to_maturity = int(input("Enter the number of years to maturity: "))
        market_rate = float(input("Enter the market interest rate (as a decimal, e.g., 0.04 for 4%): "))

        # Call the bond pricing function
        price = bond_price(face_value, coupon_rate, years_to_maturity, market_rate)

        # Print the bond price
        print(f"\nThe current bond price is: ${price:,.2f}")
    
    except ValueError:
        print("Invalid input. Please enter numerical values where required.")

# Run the main program
if __name__ == "__main__":
    main()



The current bond price is: $127,793.06


#### Portfolio analysis function that:
- Takes a list of stock prices and weights
- Calculates a portfolio return and risk
- Returns a summary dictionary

In [31]:
import yfinance as yf
import numpy as np
import pandas as pd

def portfolio_analysis(stock_tickers, weights):
    """
    Function to calculate portfolio return and risk based on stock tickers and weights.

    Parameters:
    stock_tickers (list): A list of stock tickers (e.g., ["AAPL", "MSFT"]).
    weights (list): A list of weights for each stock in the portfolio.

    Returns:
    dict: A summary dictionary containing portfolio return and risk.
    """
    # Download stock data from Yahoo Finance
    stock_data = yf.download(stock_tickers, period="1y")['Close']

    # Calculate daily returns for each stock
    stock_returns = stock_data.pct_change()

    # Calculate portfolio returns by multiplying stock returns with corresponding weights and summing them
    portfolio_returns = (stock_returns * weights).sum(axis=1)

    # Calculate portfolio risk (standard deviation of portfolio returns)
    portfolio_risk = portfolio_returns.std() * np.sqrt(252)  # Annualizing the risk (252 trading days)

    # Calculate the average annualized return for the portfolio
    portfolio_return = portfolio_returns.mean() * 252  # Annualizing the return (252 trading days)

    # Prepare the summary dictionary
    summary = {
        'Portfolio Return': portfolio_return,
        'Portfolio Risk (Standard Deviation)': portfolio_risk,
        'Individual Stock Returns': stock_returns.mean() * 252  # Annualized stock returns
    }

    return summary

# Main Program
def main():
    """
    Main program to interact with the user, retrieve stock data, and calculate portfolio metrics.
    """
    try:
        # Get user input for stock tickers and weights
        tickers_input = input("Enter the stock tickers separated by commas (e.g., AAPL, MSFT, GOOG): ")
        stock_tickers = [ticker.strip().upper() for ticker in tickers_input.split(",")]

        # Ensure the weights are provided for each stock
        weights_input = input(f"Enter the weights for each stock (e.g., 0.4, 0.3, 0.3) for {len(stock_tickers)} stocks: ")
        weights = [float(weight.strip()) for weight in weights_input.split(",")]

        if len(stock_tickers) != len(weights):
            print("Error: The number of tickers does not match the number of weights.")
            return

        # Call the portfolio analysis function
        summary = portfolio_analysis(stock_tickers, weights)

        # Display the summary
        print("\nPortfolio Analysis Summary:")
        print(f"Portfolio Return (Annualized): {summary['Portfolio Return']:.4f}")
        print(f"Portfolio Risk (Annualized Standard Deviation): {summary['Portfolio Risk (Standard Deviation)']:.4f}")
        print("\nIndividual Stock Returns (Annualized):")
        for ticker, return_rate in summary['Individual Stock Returns'].items():
            print(f"{ticker}: {return_rate:.4f}")

    except ValueError:
        print("Invalid input. Please enter numerical values for weights.")
    except Exception as e:
        print(f"Error: {e}")

# Run the main program
if __name__ == "__main__":
    main()


[*********************100%***********************]  3 of 3 completed


Portfolio Analysis Summary:
Portfolio Return (Annualized): 0.2077
Portfolio Risk (Annualized Standard Deviation): 0.1897

Individual Stock Returns (Annualized):
AAPL: 0.2820
GOOG: 0.2823
MSFT: 0.0366


#### Loan amortization function that generates a payment schedule

In [32]:
import numpy as np

def loan_amortization(principal, annual_rate, years, payments_per_year):
    """
    Function to calculate and generate a loan amortization schedule.
    
    Parameters:
    principal (float): The loan amount.
    annual_rate (float): The annual interest rate as a decimal (e.g., 5% as 0.05).
    years (int): The loan term in years.
    payments_per_year (int): The number of payments per year (e.g., 12 for monthly payments).
    
    Returns:
    list: A list containing the payment schedule with principal, interest, and balance.
    """
    # Calculate the number of payments
    total_payments = years * payments_per_year
    
    # Calculate the periodic interest rate
    rate_per_period = annual_rate / payments_per_year
    
    # Calculate the fixed monthly payment using the formula
    payment = principal * (rate_per_period * (1 + rate_per_period)**total_payments) / ((1 + rate_per_period)**total_payments - 1)
    
    # Initialize the remaining balance and a list to hold the payment schedule
    remaining_balance = principal
    payment_schedule = []

    # Loop over each period to generate the amortization schedule
    for i in range(1, total_payments + 1):
        # Interest for this period
        interest_payment = remaining_balance * rate_per_period
        
        # Principal payment for this period
        principal_payment = payment - interest_payment
        
        # Update the remaining balance
        remaining_balance -= principal_payment
        
        # Add the payment details to the schedule
        payment_schedule.append({
            'Payment Number': i,
            'Payment': round(payment, 2),
            'Principal Payment': round(principal_payment, 2),
            'Interest Payment': round(interest_payment, 2),
            'Remaining Balance': round(remaining_balance, 2)
        })

    return payment_schedule

def main():
    """
    Main program to interact with the user, retrieve loan details, and generate an amortization schedule.
    """
    try:
        # Get user input for loan details
        principal = float(input("Enter the loan amount: $"))
        annual_rate = float(input("Enter the annual interest rate (as a decimal, e.g., 0.05 for 5%): "))
        years = int(input("Enter the loan term in years: "))
        payments_per_year = int(input("Enter the number of payments per year (e.g., 12 for monthly): "))

        # Call the loan amortization function
        schedule = loan_amortization(principal, annual_rate, years, payments_per_year)

        # Print the amortization schedule
        print("\nLoan Amortization Schedule:")
        print(f"{'Payment Number':<15}{'Payment':<15}{'Principal Payment':<20}{'Interest Payment':<20}{'Remaining Balance':<20}")
        
        for payment in schedule:
            print(f"{payment['Payment Number']:<15}{payment['Payment']:<15}{payment['Principal Payment']:<20}{payment['Interest Payment']:<20}{payment['Remaining Balance']:<20}")

    except ValueError:
        print("Invalid input. Please enter numerical values where required.")
    except Exception as e:
        print(f"Unexpected error: {e}")

# Run the main program
if __name__ == "__main__":
    main()



Loan Amortization Schedule:
Payment Number Payment        Principal Payment   Interest Payment    Remaining Balance   
1              2415.74        1790.74             625.0               148209.26           
2              2415.74        1798.2              617.54              146411.06           
3              2415.74        1805.69             610.05              144605.36           
4              2415.74        1813.22             602.52              142792.15           
5              2415.74        1820.77             594.97              140971.37           
6              2415.74        1828.36             587.38              139143.02           
7              2415.74        1835.98             579.76              137307.04           
8              2415.74        1843.63             572.11              135463.41           
9              2415.74        1851.31             564.43              133612.1            
10             2415.74        1859.02             556.72     

#### Function to calculate various risk metrics (Beta, Standard Deviation, etc.)

In [36]:
import yfinance as yf
import numpy as np
import pandas as pd

def calculate_risk_metrics(stock_ticker, market_ticker, period="1y"):
    """
    Calculate various risk metrics for a given stock compared to a market index.
    
    Parameters:
        stock_ticker (str): The stock ticker symbol (e.g., "AAPL").
        market_ticker (str): The market index ticker symbol (e.g., "SPY" for S&P 500).
        period (str): The period to fetch data for (e.g., "1y" for 1 year).

    Returns:
        dict: A dictionary containing the risk metrics (Beta, Standard Deviation, and Correlation).
    """
    # Download stock and market data from Yahoo Finance (using Close prices)
    stock_data = yf.download(stock_ticker, period=period)['Close'].squeeze()
    market_data = yf.download(market_ticker, period=period)['Close'].squeeze()

    # Check if data is correctly fetched
    if stock_data.empty or market_data.empty:
        raise ValueError("No data retrieved for the tickers. Please check the tickers or try again later.")

    # Calculate daily returns for both stock and market
    stock_returns = stock_data.pct_change().dropna()
    market_returns = market_data.pct_change().dropna()

    # Check if there is enough data to calculate metrics
    if len(stock_returns) < 2 or len(market_returns) < 2:
        raise ValueError("Not enough data to calculate risk metrics. Please check the time period or tickers.")

    # Convert returns to numpy arrays for precise numerical operations
    stock_returns_arr = stock_returns.values
    market_returns_arr = market_returns.values

    # Calculate Beta: covariance(stock, market) / variance(market)
    covariance_matrix = np.cov(stock_returns_arr, market_returns_arr, ddof=1)
    covariance = covariance_matrix[0, 1]
    market_variance = np.var(market_returns_arr, ddof=1)
    beta = covariance / market_variance if market_variance != 0 else np.nan

    # Calculate Annualized Standard Deviation (Volatility) for stock returns
    stock_std_dev = np.std(stock_returns_arr, ddof=1) * np.sqrt(252)

    # Calculate Correlation between stock and market returns
    correlation_matrix = np.corrcoef(stock_returns_arr, market_returns_arr)
    correlation = correlation_matrix[0, 1]

    # Prepare the result dictionary with scalar values
    risk_metrics = {
        "Beta": float(beta),
        "Stock Standard Deviation (Annualized)": float(stock_std_dev),
        "Correlation with Market": float(correlation),
    }

    return risk_metrics

def main():
    """
    Main program to interact with the user, retrieve stock and market data, and calculate risk metrics.
    """
    try:
        # Get user input for stock and market tickers
        stock_ticker = input("Enter the stock ticker symbol (e.g., AAPL): ").upper()
        market_ticker = input("Enter the market index ticker symbol (e.g., SPY for S&P 500): ").upper()

        # Call the risk metrics calculation function
        risk_metrics = calculate_risk_metrics(stock_ticker, market_ticker)

        # Print the risk metrics using formatted floats
        print("\nRisk Metrics Summary:")
        print(f"Beta: {risk_metrics['Beta']:.4f}")
        print(f"Stock Standard Deviation (Annualized): {risk_metrics['Stock Standard Deviation (Annualized)']:.4f}")
        print(f"Correlation with Market: {risk_metrics['Correlation with Market']:.4f}")

    except ValueError as e:
        print(f"Error: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")

# Run the main program
if __name__ == "__main__":
    main()




[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Risk Metrics Summary:
Beta: 0.9174
Stock Standard Deviation (Annualized): 0.2383
Correlation with Market: 0.4885
